In [1]:
!poetry add pandas

The following packages are already present in the pyproject.toml and will be skipped:

  • pandas

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


# Import Pandas

In [2]:
import pandas as pd

## Load in Tweets

In [6]:
tweets = pd.read_json('tweets.json')

In [7]:
list_of_tweets = tweets['tweet']

Every tweet is like a dict

In [8]:
list_of_tweets[9000]

{'edit_info': {'initial': {'editTweetIds': ['1019776460633014273'],
   'editableUntil': '2018-07-19T03:20:29.402Z',
   'editsRemaining': '5',
   'isEditEligible': True}},
 'retweeted': False,
 'source': '<a href="https://transfaeries.com/" rel="nofollow">Faebot_02</a>',
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'display_text_range': ['0', '38'],
 'favorite_count': '0',
 'id_str': '1019776460633014273',
 'truncated': False,
 'retweet_count': '0',
 'id': '1019776460633014273',
 'created_at': 'Thu Jul 19 02:50:29 +0000 2018',
 'favorited': False,
 'full_text': 'E∴｡ * ･ﾟ*｡*･ ･ *ﾟ｡ * ･ ﾟ*｡･ﾟ★｡ ☆ﾟ･｡°*.',
 'lang': 'ja'}

finding a specific tweet

In [53]:
this_tweet = [ x for x in list_of_tweets if x['id_str'] == '1585439523382632448' ]

In [54]:
this_tweet

[{'edit_info': {'initial': {'editTweetIds': ['1585439523382632448'],
    'editableUntil': '2022-10-27T01:43:33.000Z',
    'editsRemaining': '5',
    'isEditEligible': True}},
  'retweeted': False,
  'source': '<a href="https://transfaeries.com/" rel="nofollow">Faebot_02</a>',
  'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
  'display_text_range': ['0', '189'],
  'favorite_count': '1',
  'id_str': '1585439523382632448',
  'truncated': False,
  'retweet_count': '0',
  'id': '1585439523382632448',
  'created_at': 'Thu Oct 27 01:13:33 +0000 2022',
  'favorited': False,
  'full_text': "biohazard: 'I try to kill myself. I had a rough couple of days, but it's alright now. I even got some sleep. Still not sure how that happened. But I'm glad it did eventually? Idk man, maybe",
  'lang': 'en'}]

The problem is this tweet was replied to, and there appears no indication of that in the metadata

In [9]:
tweets_no_replies = [ x for x in list_of_tweets if not 'in_reply_to_user_id' in x]

In [13]:
tweet_replies = [ x for x in list_of_tweets if 'in_reply_to_user_id' in x]

In [14]:
list_of_tweets.size

49663

In [15]:
len(tweets_no_replies)

48046

In [16]:
len(tweet_replies)

1617

In [9]:
type(list_of_tweets)

pandas.core.series.Series

In [10]:
type(tweets)

pandas.core.frame.DataFrame

In [20]:
retweets = [ x for x in list_of_tweets if x['retweeted'] ]

In [21]:
retweets

[]

 make the tweets full_text only and combine them

For faebot, I think I will keep faer replies

In [22]:
list_of_tweets = [x for x in list_of_tweets]

In [23]:
tweets_full_text = [x['full_text'] for x in list_of_tweets]

In [24]:
full_list_of_tweets = tweets_full_text 

In [34]:
len(full_list_of_tweets)

49663

Then dump it into a json file for later use

In [26]:
import json

In [27]:
tweets_json = json.dumps(full_list_of_tweets)
output_file = open("tweets_full.json", "w")
output_file.write(tweets_json)
output_file.close()

Trying to use the openAI data preparer tool and it wants pairs, but that should be easily doable.

the first few cells were wrongheaded attempts. 

In [28]:
# import math

In [29]:
# paired_tweets = []
# for tweet in tweets_full_text:
#     tweet_tokens = tweet.split(' ')
#     half_length = math.ceil(len(tweet_tokens)/2)
#     tweet_pair = [' '.join(tweet_tokens[:half_length]),' '.join(tweet_tokens[half_length:])]
#     paired_tweets.append(tweet_pair)

In [30]:
# tweets_json = json.dumps(paired_tweets)
# output_file = open("paired_tweets_full.json", "w")
# output_file.write(tweets_json)
# output_file.close()

This actually generates a file that openai will like.

In [32]:
output_file = open("openai_tweets_full.jsonl", "w")
for line in tweets_full_text:
    tweet_json=json.dumps({"prompt":"","completion":line})
    output_file.write(tweet_json +"\n")
output_file.close()



Let's get tweets that have been liked, retweeted, or otherwise interacted with. There doesn't seem to be a way to get replied to tweets. so let's hope they're in one of these groups.

In [36]:
 liked_tweets = [ x for x in tweets_no_replies if 'favorite_count' in x and int(x['favorite_count'])>0 ]

In [37]:
len(liked_tweets)

1552

In [39]:
retweeted_tweets= [ x for x in tweets_no_replies if 'retweet_count' in x and int(x['retweet_count'])>0 ]

In [40]:
len(retweeted_tweets)

1883

In [55]:
interacted_tweets = liked_tweets + retweeted_tweets

In [56]:
len(interacted_tweets)

3435

In [57]:
interacted_tweets_text = [ x["full_text"] for x in interacted_tweets]

In [59]:
interacted_tweets_text = [ *set(interacted_tweets_text) ]

In [60]:
len(interacted_tweets_text)

2775

In [61]:
output_file = open("openai_interacted_tweets_full.jsonl", "w")
for line in interacted_tweets_text:
    tweet_json=json.dumps({"prompt":"","completion":line})
    output_file.write(tweet_json +"\n")
output_file.close()

# Time to Fine Tune

best to run these directly in a terminal since they are interactive commands

In [62]:
! openai tools fine_tunes.prepare_data -f openai_interacted_tweets_full.jsonl 

Analyzing...

- Your file contains 2775 prompt-completion pairs
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: ^C



In [63]:
!openai api fine_tunes.create -t "openai_interacted_tweets_full_prepared.jsonl" -m davinci --suffix "Faebot02"

NameError: name 'y' is not defined